In [2]:
pip install PyPDF2 pillow pdf2image

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import threading
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk  # For progress bar
from PyPDF2 import PdfMerger
import os

class PDFMergerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PDF Merger")
        self.pdf_files = []
        self.merged_pdf_path = None

        # File upload button
        self.upload_button = tk.Button(root, text="Upload PDFs", command=self.upload_pdfs)
        self.upload_button.pack(pady=10)

        # Listbox to show uploaded PDFs
        self.file_listbox = tk.Listbox(root, selectmode=tk.SINGLE, width=50, height=10)
        self.file_listbox.pack(pady=10)

        # Up and Down buttons to reorder the PDFs
        self.up_button = tk.Button(root, text="Move Up", command=self.move_up)
        self.up_button.pack(side=tk.LEFT, padx=(50, 10))
        self.down_button = tk.Button(root, text="Move Down", command=self.move_down)
        self.down_button.pack(side=tk.RIGHT, padx=(10, 50))

        # Merge button
        self.merge_button = tk.Button(root, text="Merge PDFs", command=self.start_merge_thread)
        self.merge_button.pack(pady=10)

        # Save button
        self.save_button = tk.Button(root, text="Save Merged PDF", command=self.save_merged_pdf)
        self.save_button.pack(pady=10)

        # Status label
        self.status_label = tk.Label(root, text="Status: Ready")
        self.status_label.pack(pady=10)

        # Progress bar
        self.progress = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
        self.progress.pack(pady=10)

    def upload_pdfs(self):
        files = filedialog.askopenfilenames(
            title="Select PDF files", 
            filetypes=[("PDF files", "*.pdf")]
        )
        if files:
            self.pdf_files = list(files)
            self.file_listbox.delete(0, tk.END)
            for file in self.pdf_files:
                self.file_listbox.insert(tk.END, os.path.basename(file))

    def move_up(self):
        # Get the current selected index
        selected_idx = self.file_listbox.curselection()
        if selected_idx and selected_idx[0] > 0:
            idx = selected_idx[0]
            # Swap the positions in both listbox and pdf_files
            self.pdf_files[idx], self.pdf_files[idx - 1] = self.pdf_files[idx - 1], self.pdf_files[idx]
            self.refresh_listbox()
            self.file_listbox.select_set(idx - 1)

    def move_down(self):
        # Get the current selected index
        selected_idx = self.file_listbox.curselection()
        if selected_idx and selected_idx[0] < len(self.pdf_files) - 1:
            idx = selected_idx[0]
            # Swap the positions in both listbox and pdf_files
            self.pdf_files[idx], self.pdf_files[idx + 1] = self.pdf_files[idx + 1], self.pdf_files[idx]
            self.refresh_listbox()
            self.file_listbox.select_set(idx + 1)

    def refresh_listbox(self):
        # Clear and repopulate the listbox based on the current order of pdf_files
        self.file_listbox.delete(0, tk.END)
        for file in self.pdf_files:
            self.file_listbox.insert(tk.END, os.path.basename(file))

    def start_merge_thread(self):
        if not self.pdf_files:
            messagebox.showwarning("Warning", "Please upload PDF files first!")
            return
        
        # Start the merging process in a new thread
        self.progress["value"] = 0
        merge_thread = threading.Thread(target=self.merge_pdfs)
        merge_thread.start()

    def merge_pdfs(self):
        try:
            self.update_status("Merging PDFs...")
            merger = PdfMerger()
            total_files = len(self.pdf_files)
            for i, pdf in enumerate(self.pdf_files):
                merger.append(pdf)
                # Update the progress bar
                self.progress["value"] = ((i + 1) / total_files) * 100
                self.root.update_idletasks()
            
            # Save the merged file
            self.merged_pdf_path = "merged_output.pdf"
            with open(self.merged_pdf_path, 'wb') as f_out:
                merger.write(f_out)
            merger.close()

            self.update_status("Merging Complete!")
            messagebox.showinfo("Success", "PDFs merged successfully!")
        except Exception as e:
            self.update_status("Failed to merge PDFs")
            messagebox.showerror("Error", str(e))
        finally:
            self.progress["value"] = 0

    def save_merged_pdf(self):
        if hasattr(self, 'merged_pdf_path') and self.merged_pdf_path:
            output_file = filedialog.asksaveasfilename(
                defaultextension=".pdf", 
                filetypes=[("PDF files", "*.pdf")]
            )
            if output_file:
                try:
                    with open(self.merged_pdf_path, 'rb') as f_in:
                        with open(output_file, 'wb') as f_out:
                            f_out.write(f_in.read())
                    messagebox.showinfo("Success", f"Merged PDF saved as {output_file}")
                except Exception as e:
                    messagebox.showerror("Error", str(e))
        else:
            messagebox.showwarning("Warning", "No merged PDF available. Please merge PDFs first.")

    def update_status(self, message):
        self.status_label.config(text=f"Status: {message}")
        self.root.update_idletasks()

if __name__ == "__main__":
    root = tk.Tk()
    app = PDFMergerApp(root)
    root.mainloop()
